In [16]:
# import scenario-properties-elliptical.pkl file
import pickle
import numpy as np
from tqdm import tqdm
from utils.collisions.collisions import func_Am, func_dv, func_de


ImportError: cannot import name 'func_de' from 'utils.collisions.collisions' (c:\Users\IT\Documents\UCL\pyssem\pyssem\utils\collisions\collisions.py)

In [8]:
with open('../scenario-properties-elliptical.pkl', 'rb') as f:
    scen_properties = pickle.load(f)
all_elliptical_collision_species = scen_properties.collision_pairs

debris_species = [species for species in scen_properties.species['debris']]

# Mass Binning
binC_mass = np.zeros(len(debris_species))
binE_mass = np.zeros(2 * len(debris_species))
binW_mass = np.zeros(len(debris_species))
LBgiven = scen_properties.LC

for index, debris in enumerate(debris_species):
    binC_mass[index] = debris.mass
    binE_mass[2 * index: 2 * index + 2] = [debris.mass_lb, debris.mass_ub]
    binW_mass[index] = debris.mass_ub - debris.mass_lb

binE_mass = np.unique(binE_mass)

# Eccentricity Binning, multiple debris species will have the same eccentricity bins
binE_ecc = debris_species[0].eccentricity_bins
binE_ecc = np.sort(binE_ecc)
# Calculate the midpoints
binE_ecc = (binE_ecc[:-1] + binE_ecc[1:]) / 2
# Create bin edges starting at 0 and finishing at 1
binE_ecc = np.concatenate(([0], binE_ecc, [1]))


In [15]:
for i, species_pair in enumerate(tqdm(all_elliptical_collision_species, desc="Processing species pairs")):
    for shell_collision in tqdm(species_pair.collision_pair_by_shell, desc="Processing shell collisions", leave=False):
        # Process each shell-specific collision and append result to collision_processed list
        gamma = process_elliptical_collision_pair((i, shell_collision, scen_properties, debris_species, binE_mass, binE_ecc, LBgiven))
        species_pair.collision_processed.append(gamma)


Processing species pairs:  10%|█         | 1/10 [00:00<00:00, 80.50it/s]

0.00141372 2 1.0


NameError: name 'func_de' is not defined

In [5]:
def evolve_bins(scen_properties, m1, m2, r1, r2, v1, v2, binE_mass, binE_ecc, collision_index, n_shells=0):

    SS = 20  # super sampling ratio
    MU = 398600.4418  # km^3/s^2
    RE = 6378.1  # km

    # Ensure that m1 > m2, if not swap
    if m1 < m2:
        m1, m2 = m2, m1
        r1, r2 = r2, r1

    dv = 10
    catastrophe_ratio = (m2 * ((dv * 1000) ** 2) / (2 * m1 * 1000))  # J/g = kg*(km/s)^2 / g

    # If the specific energy is < 40 J/g: non catastrophic collision
    if catastrophe_ratio < 40:
        M = m2 * dv ** 2  # Correction from ODQN [kg*km^2/s^2]
        isCatastrophic = 0
    else:  # Catastrophic collision
        M = m1 + m2
        isCatastrophic = 1


    isCatastrophic = 0 # TESTING 

    # find the number of fragments generated
    num = (0.1 * M ** 0.75 * scen_properties.LC ** (-1.71)) - (0.1 * M ** 0.75 * min(1, 2 * r1) ** (-1.71))
    numSS = SS * num

    # Create PDF of power law distribution, then sample 'num' selections to find each mass
    dd_edges = np.logspace(np.log10(scen_properties.LC), np.log10(min(1, 2 * r1)), 500)
    dd_means = 10 ** (np.log10(dd_edges[:-1]) + np.diff(np.log10(dd_edges)) / 2)

    # Cumulative distribution
    nddcdf = 0.1 * M ** 0.75 * dd_edges ** (-1.71)
    ndd = np.maximum(0, -np.diff(nddcdf))

    # Make sure int, 0 to 1, random number for stochastic sampling of fragment diameters
    repeat_counts = np.floor(ndd).astype(int) + (np.random.rand(len(ndd)) > (1 - (ndd - np.floor(ndd)))).astype(int)
    d_pdf = np.repeat(dd_means, repeat_counts)

    try:
        dss = d_pdf[np.random.randint(0, len(d_pdf), size=int(np.ceil(numSS)))]
    except ValueError:
        dss = 0

    # Calculate the mass of objects
    A = 0.556945 * dss ** 2.0047077  # Equation 2.72
    Am = func_Am(dss, 5 if isCatastrophic == 0 else 0)  # use Am conversion of the larger object

    # order the output in descending order
    Am = np.sort(Am)[::-1]
    m = A / Am

    # Remove any fragments that are more than the original mass of m1
    m = m[m < m1]

    try:
        if max(m) == 0:
            # No fragments bigger the LNT are generated
            return None
    except ValueError:
        # Where no fragments are made
        return None
    
    # Find difference in orbital velocity for shells
    dDV = np.abs(np.median(np.diff(np.sqrt(MU / (RE + scen_properties.HMid)) * 1000)))

    # Compute delta-v for the fragments
    dv_values = func_dv(Am, 'col') / 1000  # km/s

    # Generate random directions for the velocity vectors
    u = np.random.rand(len(dv_values)) * 2 - 1
    theta = np.random.rand(len(dv_values)) * 2 * np.pi
    v = np.sqrt(1 - u ** 2)
    p = np.vstack((v * np.cos(theta), v * np.sin(theta), u)).T
    dv_vec = p * dv_values[:, np.newaxis]  # velocity vectors

    # # Calculate the new eccentricity of the fragments
    if len(m) < 100:
        a = scen_properties.HMid[collision_index] + scen_properties.re
        r_vec = np.tile([a, 0, 0], (len(m), 1))  # create an R
        v_vec = [np.linalg.norm(vec) for vec in dv_vec] + v1  # create a v vec
        v_vec_2d = np.array([[0, v, 0] for v in v_vec])
        new_ecc = func_de(r_vec, v_vec_2d)
        scen_properties = np.array([dv_vec.ravel()[:len(m)], m, new_ecc]).T  # Transpose to get the right shape (n_samples, 3)

        # Step 3: Define the bins for the 3D histogram
        binE_vel = np.arange(-n_shells, n_shells) * dDV / 1000

        zero_index = np.where(binE_vel == 0.0)[0][0]

        # Step 2: Calculate the shift to align 0 with the collision_index
        shift_amount = collision_index - zero_index

        # Step 3: Shift the array to make 0 appear at collision_index
        binE_vel_shifted = np.roll(binE_vel, shift_amount)

        # Step 4: Handle different cases
        if collision_index > n_shells // 2:
            binE_vel_selected = binE_vel_shifted[:n_shells]  # Take the first n_shells values (below 0)
        else:
            # If the collision index is less than or equal to 5, take values around 0
            start_index = max(0, collision_index - (n_shells // 2))
            end_index = start_index + n_shells  # Select the next `n_shells` values
            binE_vel_selected = binE_vel_shifted[start_index:end_index]

        bin_edges = np.zeros(len(binE_vel_selected) + 1)  # Bin edges will be one more than selected values

        # Calculate the midpoints between consecutive bin values to generate the edges
        bin_edges[1:-1] = (binE_vel_selected[:-1] + binE_vel_selected[1:]) / 2
        # For the first and last bin edge, extrapolate slightly beyond the min/max
        bin_edges[0] = binE_vel_selected[0] - (binE_vel_selected[1] - binE_vel_selected[0]) / 2
        bin_edges[-1] = binE_vel_selected[-1] + (binE_vel_selected[-1] - binE_vel_selected[-2]) / 2

        # Slice the array to get the values within the desired range
        bins = [bin_edges, binE_mass, binE_ecc]

        # Step 4: Use np.histogramdd to create the 3D histogram
        hist, edges = np.histogramdd(scen_properties, bins=bins)

        hist = hist/SS

        # Sum over the velocity bins to collapse the 3D histogram into a 2D histogram

        return hist

    else:
        # Bin the velocities
        def func_de_single(R, V):
            mu = 398600.4418  # km^3/s^2
            V = np.abs(V)
            h = np.cross(R, V)
            e_vector = (np.cross(V, h) / mu) - (R / np.linalg.norm(R))
            e = np.linalg.norm(e_vector)
            return e

        # Flatten dv_vec
        flattened_dv_vec = dv_vec.ravel()

        # Take only the values up to the length of m
        sliced_dv_vec = flattened_dv_vec[:len(m)]

        # Define the number of bins
        num_bins = 100  # Adjust this number based on your needs

        # Bin the velocities
        dv_bins = np.linspace(np.min(sliced_dv_vec), np.max(sliced_dv_vec), num_bins)
        # bin the masses
        m_bins = np.linspace(np.min(m), np.max(m), num_bins)

        # Create a single R vector
        a = scen_properties.HMid[collision_index] + scen_properties.re
        r_vec = np.array([a, 0, 0])  # create a single R

        # create new v vectors
        v_vec = [np.linalg.norm(vec) for vec in dv_bins] + v1  # create a v vec

        # Compute eccentricities for each bin
        new_ecc = [func_de_single(r_vec, np.array([0, v, 0])) for v in v_vec]

        # Combine the scen_properties into a single array and transpose to get the right shape (n_samples, 3)
        scen_properties = np.array([dv_bins, m_bins, new_ecc]).T

        # Step 3: Define the bins for the 3D histogram
        binE_vel = np.arange(-n_shells, n_shells) * dDV / 1000

        zero_index = np.where(binE_vel == 0.0)[0][0]

        # Step 2: Calculate the shift to align 0 with the collision_index
        shift_amount = collision_index - zero_index

        # Step 3: Shift the array to make 0 appear at collision_index
        binE_vel_shifted = np.roll(binE_vel, shift_amount)

        # Step 4: Handle different cases
        if collision_index > n_shells // 2:
            # If the collision index is greater than 5, take the bottom values (below 0)
            binE_vel_selected = binE_vel_shifted[:n_shells]  # Take the first n_shells values (below 0)
        else:
            # If the collision index is less than or equal to 5, take values around 0
            start_index = max(0, collision_index - (n_shells // 2))
            end_index = start_index + n_shells  # Select the next `n_shells` values
            binE_vel_selected = binE_vel_shifted[start_index:end_index]

        bin_edges = np.zeros(len(binE_vel_selected) + 1)  # Bin edges will be one more than selected values

        # Calculate the midpoints between consecutive bin values to generate the edges
        bin_edges[1:-1] = (binE_vel_selected[:-1] + binE_vel_selected[1:]) / 2
        # For the first and last bin edge, extrapolate slightly beyond the min/max
        bin_edges[0] = binE_vel_selected[0] - (binE_vel_selected[1] - binE_vel_selected[0]) / 2
        bin_edges[-1] = binE_vel_selected[-1] + (binE_vel_selected[-1] - binE_vel_selected[-2]) / 2

        # Slice the array to get the values within the desired range
        bins = [bin_edges, binE_mass, binE_ecc]

        # Use np.histogramdd to create the 3D histogram
        hist, edges = np.histogramdd(scen_properties, bins=bins)

        # Normalize the histogram
        hist = hist / SS

        return hist

In [6]:
def process_elliptical_collision_pair(args):
    """
    A similar function to the process_species_pair, apart from it as the shells are already defined and the velocity, 
    you are able to calculate evolve bins just once. 

    """
    i, collision_pair, scen_properties, debris_species, binE_mass, binE_ecc, LBgiven = args
    m1, m2 = collision_pair.species1.mass, collision_pair.species2.mass
    r1, r2 = collision_pair.species1.radius, collision_pair.species2.radius


    # there needs to be some changes here to account for the fact that the shells are already defined
    # set fragment_spreading to True
    v1 = collision_pair.species1.velocity_per_shells[collision_pair.shell_index][collision_pair.shell_index]
    v2 = collision_pair.species2.velocity_per_shells[collision_pair.shell_index][collision_pair.shell_index]

    # This the time per shell for each species - the output still assumes that it is always there, so you need to divide
    t1 = collision_pair.species1.time_per_shells[collision_pair.shell_index][collision_pair.shell_index]
    t2 = collision_pair.species2.time_per_shells[collision_pair.shell_index][collision_pair.shell_index]
    min_TIS = min(t1, t2)
    print(m1, m2, min_TIS)

    fragments = evolve_bins(scen_properties, m1, m2, r1, r2, v1, v2, binE_mass, binE_ecc, collision_pair.shell_index, n_shells=scen_properties.n_shells)    
    
    if fragments is not None:
        collision_pair.fragments = fragments # * min_TIS
    else: 
        collision_pair.fragments = fragments

    return collision_pair